In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI


load_dotenv()


model = ChatOpenAI(
    model=os.environ.get("MODEL_NAME"),
    temperature=0,
    base_url=os.environ.get("COMPATIBLE_BASE_URL"),
    api_key=os.environ.get("COMPATIBLE_API_KEY"),
)

In [2]:
response = model.invoke("hi")
response

AIMessage(content='Hello! How can I help you today? 😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 9, 'total_tokens': 20, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'qwen3-max', 'system_fingerprint': None, 'id': 'chatcmpl-0c454f65-3908-47c6-926e-195be796b782', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--fdd120a8-81d0-4daa-a9db-e9eae7f42c66-0', usage_metadata={'input_tokens': 9, 'output_tokens': 11, 'total_tokens': 20, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

In [5]:
for chunk in model.stream("hi"):
    print(chunk.content, end="|", flush=True)

|Hello|! How| can I help| you today?| 😊||

In [7]:
full = None  # None | AIMessageChunk
for chunk in model.stream("Hi"):
    full = chunk if full is None else full + chunk

full

AIMessageChunk(content='Hello! How can I help you today? 😊', additional_kwargs={}, response_metadata={'model_provider': 'openai', 'finish_reason': 'stop', 'model_name': 'qwen3-max'}, id='lc_run--9d90c1ad-6f04-45af-ac4b-8769854140ed', chunk_position='last')

In [8]:
responses = model.batch([
    "hi",
    "你好",
    "安徽省省会在哪？"
])

for response in responses:
    print(response)

content='Hello! How can I help you today? 😊' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 9, 'total_tokens': 20, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'qwen3-max', 'system_fingerprint': None, 'id': 'chatcmpl-608dc86a-4ff3-4d86-b001-ed0dd0254b64', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--d6d282ed-a748-47c8-9197-a30fa4ee4ce9-0' usage_metadata={'input_tokens': 9, 'output_tokens': 11, 'total_tokens': 20, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}}
content='你好！有什么我可以帮你的吗？😊' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 9, 'total_tokens': 18, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'qwen3-max', 'system_finger

In [9]:
for response in model.batch_as_completed([
    "hi",
    "你好",
    "安徽省省会在哪？"
]):
    print(response)

(2, AIMessage(content='安徽省的省会是合肥市。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 13, 'total_tokens': 20, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'qwen3-max', 'system_fingerprint': None, 'id': 'chatcmpl-7ccd149c-f78b-4a3a-9165-4ab627b2f45f', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--ca6a770b-7929-4b58-ad01-a2d92c15a96e-0', usage_metadata={'input_tokens': 13, 'output_tokens': 7, 'total_tokens': 20, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}}))
(1, AIMessage(content='你好！很高兴见到你，有什么我可以帮忙的吗？😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 9, 'total_tokens': 22, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'qwen3

In [ ]:
from pydantic import BaseModel, Field
from langchain.agents import create_agent


class ContactInfo(BaseModel):
    """Contact information for a person."""
    name: str = Field(description="The name of the person")
    email: str = Field(description="The email address of the person")
    phone: str = Field(description="The phone number of the person")

agent = create_agent(
    model="gpt-5",
    tools=[],
    response_format=ContactInfo  # Auto-selects ProviderStrategy
)

result = agent.invoke({
    "messages": [{"role": "user", "content": "Extract contact info from: John Doe, john@example.com, (555) 123-4567"}]
})

result["structured_response"]

ContactInfo(name='John Doe', email='john@example.com', phone='(555) 123-4567')

In [13]:
agent = create_agent(
    model=model,
    tools=[],
    response_format=ContactInfo
)

result = agent.invoke({
    "messages": [{"role": "user", "content": "Extract contact info from: John Doe, john@example.com, (555) 123-4567"}]
})

result["structured_response"]

ContactInfo(name='John Doe', email='john@example.com', phone='(555) 123-4567')

In [15]:
from langchain.agents.structured_output import ToolStrategy


qwen3_32b = ChatOpenAI(
    model="qwen3-32b",
    temperature=0,
    base_url=os.environ.get("COMPATIBLE_BASE_URL"),
    api_key=os.environ.get("COMPATIBLE_API_KEY"),
    extra_body={"enable_thinking": False},
)

agent = create_agent(
    model=qwen3_32b,
    tools=[],
    response_format=ToolStrategy(ContactInfo)
)

result = agent.invoke({
    "messages": [{"role": "user", "content": "Extract contact info from: John Doe, john@example.com, (555) 123-4567"}]
})

result["structured_response"]

ContactInfo(name='John Doe', email='john@example.com', phone='(555) 123-4567')

In [17]:
gpt5 = ChatOpenAI(model="gpt-5", temperature=0)
gpt5_with_structure = gpt5.with_structured_output(ContactInfo)
gpt5_with_structure.invoke("Extract contact info from: John Doe, john@example.com, (555) 123-4567")

ContactInfo(name='John Doe', email='john@example.com', phone='(555) 123-4567')

In [18]:
model_with_structure = model.with_structured_output(ContactInfo)
model_with_structure.invoke("Extract contact info from: John Doe, john@example.com, (555) 123-4567")

BadRequestError: Error code: 400 - {'error': {'code': 'invalid_parameter_error', 'param': None, 'message': "<400> InternalError.Algo.InvalidParameter: 'messages' must contain the word 'json' in some form, to use 'response_format' of type 'json_object'.", 'type': 'invalid_request_error'}, 'id': 'chatcmpl-92acbeb9-e868-47c6-aae7-14953690dd55', 'request_id': '92acbeb9-e868-47c6-aae7-14953690dd55'}

In [19]:
model_with_structure = model.with_structured_output(ContactInfo)
model_with_structure.invoke("Extract contact info in JSON format from: John Doe, john@example.com, (555) 123-4567")

ContactInfo(name='John Doe', email='john@example.com', phone='(555) 123-4567')